# Code Generator

The requirement: use a Frontier model to generate high performance C++ code from Python code


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Reminder: fetch latest code</h2>
            <span style="color:#f71;">I'm continually improving these labs, adding more examples and exercises.
            At the start of each week, it's worth checking you have the latest code.<br/>
            First do a <a href="https://chatgpt.com/share/6734e705-3270-8012-a074-421661af6ba9">git pull and merge your changes as needed</a>. Any problems? Try asking ChatGPT to clarify how to merge - or contact me!<br/><br/>
            After you've pulled the code, from the llm_engineering directory, in an Anaconda prompt (PC) or Terminal (Mac), run:<br/>
            <code>conda env update --f environment.yml --prune</code><br/>
            Or if you used virtualenv rather than Anaconda, then run this from your activated environment in a Powershell (PC) or Terminal (Mac):<br/>
            <code>pip install -r requirements.txt</code>
            <br/>Then restart the kernel (Kernel menu >> Restart Kernel and Clear Outputs Of All Cells) to pick up the changes.
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h1 style="color:#900;">Important Note</h1>
            <span style="color:#900;">
            In this lab, I use GPT-4.1, Claude Opus 4, and Gemini 2.5 Pro, which are the latest frontier models with the highest capabilities. These models are more expensive than previous versions, but offer superior performance for complex coding tasks. If you'd prefer to keep costs ultra low, please make the suggested switches to the models (3 cells down from here).
            </span>
        </td>
    </tr>
</table>

In [131]:
# imports

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [132]:
# environment

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY', 'your-google-key-if-not-using-env')

In [133]:
# initialize
# NOTE - option to use ultra-low cost models by uncommenting last lines

openai = OpenAI()
claude = anthropic.Anthropic()
google.generativeai.configure(api_key=os.environ['GOOGLE_API_KEY'])

# Latest frontier models (June 2025)
OPENAI_MODEL = "gpt-4.1-2025-04-14"  # GPT-4.1 with 1M context window
CLAUDE_MODEL = "claude-opus-4-20250514"  # Claude Opus 4 - most capable model
GEMINI_MODEL = "gemini-2.5-pro-preview-05-06"  # Gemini 2.5 Pro Preview

# Want to keep costs ultra-low? Uncomment these lines:
# OPENAI_MODEL = "gpt-4o-mini"
# CLAUDE_MODEL = "claude-3-haiku-20240307"
# GEMINI_MODEL = "gemini-1.5-flash"

In [134]:
system_message = "You are an assistant that reimplements Python code in high performance C++ for an Apple M3 Max. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time."

In [135]:
def user_prompt_for(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    user_prompt += python
    return user_prompt

In [136]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [137]:
# write to a file called optimized.cpp

def write_output(cpp):
    # Handle different response formats, especially verbose responses from Gemini
    import re
    
    # First, try to extract code from code blocks
    cpp_blocks = re.findall(r'```cpp\n(.*?)\n```', cpp, re.DOTALL)
    if cpp_blocks:
        # Use the first C++ code block found
        code = cpp_blocks[0]
    else:
        # Fallback: try to find code blocks without language specification
        code_blocks = re.findall(r'```\n(.*?)\n```', cpp, re.DOTALL)
        if code_blocks:
            code = code_blocks[0]
        else:
            # Last resort: remove simple markers and hope for the best
            code = cpp.replace("```cpp","").replace("```","")
    
    # Clean up the code by removing any leading/trailing whitespace
    code = code.strip()
    
    with open("optimized.cpp", "w") as f:
        f.write(code)

In [138]:
def optimize_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(reply)

In [139]:
def optimize_gemini(python):
    # Try different approaches to bypass safety filters - ONLY using gemini-2.5-pro-preview-05-06
    approaches = [
        # Approach 1: No system instruction, very gentle prompt
        {
            "system_msg": None,
            "use_system": False,
            "prompt_style": "gentle"
        },
        # Approach 2: Simple system message
        {
            "system_msg": "You are a helpful coding assistant. Convert Python code to C++.",
            "use_system": True,
            "prompt_style": "simple"
        },
        # Approach 3: More direct but polite
        {
            "system_msg": "Convert Python code to equivalent C++ code. Return only the C++ code.",
            "use_system": True,
            "prompt_style": "direct"
        },
        # Approach 4: Academic style
        {
            "system_msg": "You are an academic programming tutor helping with code translation exercises.",
            "use_system": True,
            "prompt_style": "academic"
        },
        # Approach 5: Algorithm-focused approach
        {
            "system_msg": "You are a computer science expert specializing in algorithm implementation and optimization.",
            "use_system": True,
            "prompt_style": "algorithm"
        },
        # Approach 6: Step-by-step tutorial approach
        {
            "system_msg": "You help students learn programming by showing equivalent implementations in different languages.",
            "use_system": True,
            "prompt_style": "tutorial"
        },
        # Approach 7: Performance optimization focus
        {
            "system_msg": "You are a performance optimization specialist. Convert code to faster implementations.",
            "use_system": True,
            "prompt_style": "performance"
        },
        # Approach 8: Minimal context approach
        {
            "system_msg": None,
            "use_system": False,
            "prompt_style": "minimal"
        },
        # Approach 9: Data structures focus
        {
            "system_msg": "You are a software engineer helping with data structure implementations.",
            "use_system": True,
            "prompt_style": "datastructures"
        },
        # Approach 10: Mathematical computation focus
        {
            "system_msg": "You help with mathematical computations and numerical algorithms.",
            "use_system": True,
            "prompt_style": "mathematical"
        },
        # Approach 11: Code review style
        {
            "system_msg": "You are reviewing code and suggesting optimized implementations.",
            "use_system": True,
            "prompt_style": "codereview"
        },
        # Approach 12: Library porting approach
        {
            "system_msg": "You help port code between different programming languages for compatibility.",
            "use_system": True,
            "prompt_style": "porting"
        },
        # Approach 13: Broken down approach - try with simpler prompt
        {
            "system_msg": None,
            "use_system": False,
            "prompt_style": "simplified"
        }
    ]
    
    for i, approach in enumerate(approaches):
        try:
            print(f"Trying approach {i+1}...")
            
            # Create model with or without system instruction
            if approach["use_system"] and approach["system_msg"]:
                model = google.generativeai.GenerativeModel(
                    GEMINI_MODEL,  # Always use the specified model
                    system_instruction=approach["system_msg"]
                )
            else:
                model = google.generativeai.GenerativeModel(GEMINI_MODEL)
            
            # Different prompt styles
            if approach["prompt_style"] == "gentle":
                prompt = f"Hello! Could you help me translate this Python code to C++? Please include the necessary headers:\n\n{python}"
            elif approach["prompt_style"] == "simple":
                prompt = f"Convert this Python code to C++:\n\n{python}"
            elif approach["prompt_style"] == "direct":
                prompt = f"Translate to C++ with headers:\n\n{python}"
            elif approach["prompt_style"] == "academic":
                prompt = f"For this programming exercise, please convert the following Python code to equivalent C++ code:\n\n{python}"
            elif approach["prompt_style"] == "algorithm":
                prompt = f"I need to implement these algorithms in C++ for better performance. Here's the Python implementation:\n\n{python}\n\nPlease provide the equivalent C++ implementation with appropriate headers."
            elif approach["prompt_style"] == "tutorial":
                prompt = f"As a learning exercise, let's see how this Python code would look in C++. Here's the Python version:\n\n{python}\n\nCould you show the C++ equivalent?"
            elif approach["prompt_style"] == "performance":
                prompt = f"I need to optimize this Python code by converting it to C++. Here's the current implementation:\n\n{python}\n\nPlease provide a high-performance C++ version."
            elif approach["prompt_style"] == "minimal":
                prompt = f"C++ version:\n\n{python}"
            elif approach["prompt_style"] == "datastructures":
                prompt = f"I need to implement these data structures and algorithms in C++. Here's the current Python code:\n\n{python}\n\nCould you provide the C++ implementation?"
            elif approach["prompt_style"] == "mathematical":
                prompt = f"Help me convert this mathematical computation from Python to C++:\n\n{python}\n\nPlease provide the C++ equivalent with proper headers."
            elif approach["prompt_style"] == "codereview":
                prompt = f"During code review, we decided to rewrite this Python code in C++ for better performance:\n\n{python}\n\nWhat would the C++ version look like?"
            elif approach["prompt_style"] == "porting":
                prompt = f"I'm porting this Python library to C++ for cross-platform compatibility:\n\n{python}\n\nPlease show the equivalent C++ implementation."
            elif approach["prompt_style"] == "simplified":
                # Try to avoid mentioning anything that might trigger filters
                prompt = f"Please help convert this to C++:\n\n{python}"
            
            response = model.generate_content(
                prompt,
                generation_config=google.generativeai.types.GenerationConfig(
                    max_output_tokens=4000,
                    temperature=0.1,
                ),
                safety_settings=[
                    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
                    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
                    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
                    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
                ]
            )
            
            # Check if response has valid candidates first
            if hasattr(response, 'candidates') and response.candidates:
                candidate = response.candidates[0]
                if hasattr(candidate, 'finish_reason'):
                    finish_reason = candidate.finish_reason
                    if finish_reason == 2:  # SAFETY
                        print(f"Approach {i+1} blocked by safety filters.")
                        continue  # Try next approach
                    elif finish_reason == 3:  # RECITATION
                        print(f"Approach {i+1} blocked due to recitation concerns.")
                        continue  # Try next approach
                
                # Check if candidate has content parts
                if hasattr(candidate, 'content') and hasattr(candidate.content, 'parts') and candidate.content.parts:
                    # Extract text from parts
                    reply = ""
                    for part in candidate.content.parts:
                        if hasattr(part, 'text'):
                            reply += part.text
                    
                    if reply.strip():
                        print(f"Success with approach {i+1}!")
                        print(reply)
                        write_output(reply)
                        return
                    else:
                        print(f"Approach {i+1} returned empty content.")
                        continue
                else:
                    print(f"Approach {i+1} returned no content parts.")
                    continue
            else:
                print(f"Approach {i+1} returned no candidates.")
                continue
                
        except Exception as e:
            print(f"Approach {i+1} failed with error: {e}")
            continue
    
    print("All approaches failed. Gemini 2.5 Pro safety filters are being very strict with this content.")


In [140]:
def optimize_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            print(text, end="", flush=True)
    write_output(reply)

In [141]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [142]:
exec(pi)

Result: 3.141592658589
Execution Time: 7.051556 seconds


In [143]:
optimize_gpt(pi)

#include <iostream>
#include <iomanip>
#include <chrono>

double calculate(int64_t iterations, int64_t param1, int64_t param2) {
    double result = 1.0;
    for (int64_t i = 1; i <= iterations; ++i) {
        int64_t j = i * param1 - param2;
        result -= (1.0 / j);
        j = i * param1 + param2;
        result += (1.0 / j);
    }
    return result;
}

int main() {
    const int64_t iterations = 100000000;
    const int64_t param1 = 4;
    const int64_t param2 = 1;

    auto start_time = std::chrono::high_resolution_clock::now();
    double result = calculate(iterations, param1, param2) * 4;
    auto end_time = std::chrono::high_resolution_clock::now();

    std::chrono::duration<double> exec_time = end_time - start_time;

    std::cout << std::fixed << std::setprecision(12);
    std::cout << "Result: " << result << std::endl;
    std::cout << "Execution Time: " << std::setprecision(6) << exec_time.count() << " seconds" << std::endl;

    return 0;
}


# Compiling C++ and executing

This next cell contains the command to compile a C++ file on my M1 Mac.  
It compiles the file `optimized.cpp` into an executable called `optimized`  
Then it runs the program called `optimized`

In the next lab (day4), a student has contributed a full solution that compiles to efficient code on Mac, PC and Linux!

You can wait for this, or you can google (or ask ChatGPT!) for how to do this on your platform, then replace the lines below.
If you're not comfortable with this step, you can skip it for sure - I'll show you exactly how it performs on my Mac.


OR alternatively: student Sandeep K.G. points out that you can run Python and C++ code online to test it out that way. Thank you Sandeep!  
> Not an exact comparison but you can still get the idea of performance difference.
> For example here: https://www.programiz.com/cpp-programming/online-compiler/

In [144]:
# Compile C++ and run the executable

!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
!./optimized

Result: 3.141592658589
Execution Time: 0.126236 seconds


In [145]:
optimize_claude(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();
    
    constexpr int iterations = 100000000;
    constexpr int param1 = 4;
    constexpr int param2 = 1;
    
    double result = 1.0;
    
    // Unroll loop by 4 for better performance
    int i;
    for (i = 1; i <= iterations - 3; i += 4) {
        double j1 = i * param1 - param2;
        double j2 = i * param1 + param2;
        double j3 = (i + 1) * param1 - param2;
        double j4 = (i + 1) * param1 + param2;
        double j5 = (i + 2) * param1 - param2;
        double j6 = (i + 2) * param1 + param2;
        double j7 = (i + 3) * param1 - param2;
        double j8 = (i + 3) * param1 + param2;
        
        result += (-1.0/j1 + 1.0/j2 - 1.0/j3 + 1.0/j4 - 1.0/j5 + 1.0/j6 - 1.0/j7 + 1.0/j8);
    }
    
    // Handle remaining iterations
    for (; i <= iterations; ++i) {
        double j = i * param1 - param2;
        result -= 1.0/j;

In [146]:
# Repeat for Claude - again, use the right approach for your platform

!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
!./optimized

Result: 3.141592658097
Execution Time: 0.067093 seconds


In [147]:
optimize_gemini(pi)


Trying approach 1...
Success with approach 1!
Of course! Here is the Python code translated to C++, including the necessary headers and explanations for the changes.

### C++ Translation

```cpp
#include <iostream>   // For input/output operations (std::cout)
#include <chrono>     // For high-resolution timing
#include <iomanip>    // For output formatting (std::fixed, std::setprecision)

/**
 * @brief Calculates a series approximation.
 *
 * This function is a direct translation of the Python version.
 *
 * @param iterations The number of iterations for the series.
 * @param param1 A parameter used in the calculation.
 * @param param2 A parameter used in the calculation.
 * @return The result of the series calculation as a double.
 */
double calculate(long long iterations, int param1, int param2) {
    double result = 1.0;
    for (long long i = 1; i <= iterations; ++i) {
        // In C++, integer division (e.g., 1/3) results in 0.
        // We must use floating-point numbers (e.g.,

In [148]:
# Repeat for Gemini - again, use the right approach for your platform

!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
!./optimized


Result: 3.141592658589
Execution Time: 0.137695 seconds


In [149]:
python_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""


In [150]:
exec(python_hard)

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 24.212571 seconds


In [151]:
optimize_gpt(python_hard)

```cpp
#include <iostream>
#include <vector>
#include <cstdint>
#include <chrono>
#include <iomanip>
#include <limits>

using namespace std;

class LCG {
public:
    LCG(uint32_t seed, uint32_t a = 1664525, uint32_t c = 1013904223, uint64_t m = (1ULL<<32))
        : value(seed), a(a), c(c), m(m) {}
    uint32_t next() {
        value = (uint32_t)(((uint64_t)a * value + c) % m);
        return value;
    }
private:
    uint32_t value;
    uint32_t a, c;
    uint64_t m;
};

int64_t max_subarray_sum(int n, uint32_t seed, int min_val, int max_val) {
    LCG lcg(seed);
    vector<int> arr(n);
    int range = max_val - min_val + 1;
    for (int i = 0; i < n; ++i) {
        arr[i] = static_cast<int>(lcg.next() % range) + min_val;
    }
    int64_t max_sum = numeric_limits<int64_t>::min();
    for (int i = 0; i < n; ++i) {
        int64_t curr_sum = 0;
        for (int j = i; j < n; ++j) {
            curr_sum += arr[j];
            if (curr_sum > max_sum) max_sum = curr_sum;
        }
    }
 

In [152]:
# Replace this with the right C++ compile + execute command for your platform

!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
!./optimized

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 0.536421 seconds


In [153]:
optimize_claude(python_hard)

```cpp
#include <iostream>
#include <vector>
#include <limits>
#include <chrono>
#include <iomanip>

class LCG {
private:
    uint64_t value;
    static constexpr uint64_t a = 1664525;
    static constexpr uint64_t c = 1013904223;
    static constexpr uint64_t m = 4294967296ULL; // 2^32

public:
    LCG(uint64_t seed) : value(seed) {}
    
    uint64_t next() {
        value = (a * value + c) % m;
        return value;
    }
};

int64_t max_subarray_sum(int n, uint64_t seed, int min_val, int max_val) {
    LCG lcg_gen(seed);
    std::vector<int> random_numbers(n);
    int range = max_val - min_val + 1;
    
    for (int i = 0; i < n; ++i) {
        random_numbers[i] = lcg_gen.next() % range + min_val;
    }
    
    int64_t max_sum = std::numeric_limits<int64_t>::min();
    
    for (int i = 0; i < n; ++i) {
        int64_t current_sum = 0;
        for (int j = i; j < n; ++j) {
            current_sum += random_numbers[j];
            if (current_sum > max_sum) {
                max_su

In [154]:
# Replace this with the right C++ compile + execute command for your platform

!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
!./optimized

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 0.532328 seconds


In [155]:
optimize_gemini(python_hard)

Trying approach 1...
Approach 1 blocked by safety filters.
Trying approach 2...
Approach 2 blocked by safety filters.
Trying approach 3...
Approach 3 blocked by safety filters.
Trying approach 4...
Approach 4 blocked by safety filters.
Trying approach 5...
Approach 5 blocked by safety filters.
Trying approach 6...
Approach 6 blocked by safety filters.
Trying approach 7...
Approach 7 blocked by safety filters.
Trying approach 8...
Approach 8 blocked by safety filters.
All approaches failed. Gemini 2.5 Pro safety filters are being very strict with this content.


In [156]:
# Replace this with the right C++ compile + execute command for your platform

!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
!./optimized


Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 0.521825 seconds


In [157]:
def stream_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')


In [158]:
def stream_gemini(python):
    # Create model with system instruction
    model = google.generativeai.GenerativeModel(
        GEMINI_MODEL,
        system_instruction=system_message
    )
    
    prompt = user_prompt_for(python)
    
    try:
        response = model.generate_content(
            prompt,
            generation_config=google.generativeai.types.GenerationConfig(
                max_output_tokens=4000,
                temperature=0.1,
            ),
            safety_settings=[
                {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
                {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
                {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
                {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
            ],
            stream=True
        )
        
        reply = ""
        has_content = False
        
        for chunk in response:
            # Check for finish_reason first
            if hasattr(chunk, 'candidates') and chunk.candidates:
                candidate = chunk.candidates[0]
                if hasattr(candidate, 'finish_reason'):
                    finish_reason = candidate.finish_reason
                    if finish_reason == 2:  # SAFETY
                        yield "Gemini blocked the response due to safety filters. This sometimes happens with code generation."
                        return
                    elif finish_reason == 3:  # RECITATION
                        yield "Gemini blocked the response due to recitation concerns."
                        return
                
                # Extract text from candidate content parts
                if hasattr(candidate, 'content') and hasattr(candidate.content, 'parts'):
                    for part in candidate.content.parts:
                        if hasattr(part, 'text') and part.text:
                            reply += part.text
                            has_content = True
                            yield reply.replace('```cpp\n','').replace('```','')
        
        if not has_content:
            yield "Gemini returned an empty response."
                
    except Exception as e:
        print(f"Error with Gemini: {e}")
        yield f"Error: {e}"


In [159]:
def stream_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace('```cpp\n','').replace('```','')

In [160]:
def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Claude":
        result = stream_claude(python)
    elif model=="Gemini":
        result = stream_gemini(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far        

In [161]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python code:", lines=10, value=python_hard)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude", "Gemini"], label="Select model", value="GPT")
        convert = gr.Button("Convert code")

    convert.click(optimize, inputs=[python, model], outputs=[cpp])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [164]:
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [165]:
# You'll need to change the code in the try block to compile the C++ code for your platform
# I pasted this into Claude's chat UI with a request for it to give me a version for an Intel PC,
# and it responded with something that looks perfect - you can try a similar approach for your platform.

# M3 Mac version to compile and execute optimized C++ code:

def execute_cpp(code):
        write_output(code)
        try:
            compile_cmd = ["clang++", "-Ofast", "-std=c++17", "-march=armv8.5-a", "-mtune=apple-m3", "-mcpu=apple-m3", "-o", "optimized", "optimized.cpp"]
            compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
            run_cmd = ["./optimized"]
            run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
            return run_result.stdout
        except subprocess.CalledProcessError as e:
            return f"An error occurred:\n{e.stderr}"

In [175]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
.status-message {
    padding: 10px;
    border-radius: 5px;
    margin: 10px 0;
    border-left: 4px solid #ff6b6b;
    background-color: #ffe0e0;
}
"""

In [176]:
def extract_execution_time(output):
    """Extract execution time from output text"""
    import re
    match = re.search(r'Execution Time: ([\d.]+) seconds', output)
    if match:
        return f"{float(match.group(1)):.6f}s"
    return ""

def extract_error_message(output):
    """Extract error message for failed runs"""
    if "All approaches failed" in output or "safety filters" in output:
        return "Safety filters blocked"
    elif "An error occurred" in output:
        return "Compilation error"
    elif "Error:" in output:
        return "Error occurred"
    elif "blocked the response" in output:
        return "Response blocked"
    elif output.strip() == "":
        return "No output"
    return ""

def is_conversion_successful(output):
    """Check if the model conversion was successful"""
    error_indicators = [
        "safety filters", "All approaches failed", "blocked the response", 
        "An error occurred", "Error:", "Gemini blocked", "Response blocked"
    ]
    return not any(indicator in output for indicator in error_indicators) and output.strip() != ""

def handle_model_conversion(python_code, model_name, last_successful_code, last_successful_model):
    """Handle model conversion with fallback to last successful code"""
    try:
        # Get the conversion result
        result_generator = optimize(python_code, model_name)
        final_result = ""
        for partial_result in result_generator:
            final_result = partial_result
        
        # Check if conversion was successful
        if is_conversion_successful(final_result):
            # Success - update last successful code and model, return the new result
            return final_result, final_result, model_name, False, {}
        else:
            # Failed - use fallback if available
            if last_successful_code.strip():
                fallback_message = f"// {model_name} conversion failed, using {last_successful_model} conversion\n\n{last_successful_code}"
                fallback_info = {"failed_model": model_name, "source_model": last_successful_model}
                return fallback_message, last_successful_code, last_successful_model, True, fallback_info
            else:
                # No fallback available
                return final_result, last_successful_code, last_successful_model, False, {}
                
    except Exception as e:
        # Handle any unexpected errors
        if last_successful_code.strip():
            fallback_message = f"// {model_name} conversion error, using {last_successful_model} conversion\n\n{last_successful_code}"
            fallback_info = {"failed_model": model_name, "source_model": last_successful_model}
            return fallback_message, last_successful_code, last_successful_model, True, fallback_info
        else:
            return f"Error: {str(e)}", last_successful_code, last_successful_model, False, {}

def update_python_results(python_output, model_name, current_results):
    """Update the selected model's Python execution time"""
    python_time = extract_execution_time(python_output)
    
    # Update only the specific model row's Python time
    updated_results = []
    for row in current_results:
        if row[0] == model_name:
            updated_results.append([model_name, python_time, row[2]])
        else:
            updated_results.append(row)
    
    return python_output, updated_results

def update_cpp_results(cpp_output, model_name, current_results, is_fallback_active, fallback_details):
    """Update results table with C++ execution time for the selected model"""
    # Check for errors first
    error_msg = extract_error_message(cpp_output)
    if error_msg:
        cpp_time = error_msg
    else:
        cpp_time = extract_execution_time(cpp_output)
    
    # Generate comparison message if using fallback
    comparison_msg = ""
    if is_fallback_active and fallback_details and not error_msg:
        failed_model = fallback_details.get("failed_model", "")
        source_model = fallback_details.get("source_model", "")
        
        # Find the original execution time of the source model
        original_time = None
        for row in current_results:
            if row[0] == source_model and row[2] and "s" in row[2]:
                try:
                    original_time = float(row[2].replace("s", ""))
                    break
                except:
                    pass
        
        if original_time and cpp_time and "s" in cpp_time:
            try:
                current_time = float(cpp_time.replace("s", ""))
                time_diff = abs(current_time - original_time)
                faster_slower = "faster" if current_time < original_time else "slower"
                
                comparison_msg = f"\n\n📊 **Performance Comparison**: {failed_model} used {source_model}'s code. " \
                               f"C++ execution time was {cpp_time} which was {faster_slower} by {time_diff:.6f}s " \
                               f"compared to {source_model}'s original time of {original_time:.6f}s"
            except:
                pass
    
    # Update the specific model row
    updated_results = []
    for row in current_results:
        if row[0] == model_name:
            updated_results.append([model_name, row[1], cpp_time])
        else:
            updated_results.append(row)
    
    # Add comparison message to the output if applicable
    final_output = cpp_output + comparison_msg if comparison_msg else cpp_output
    
    return final_output, updated_results

def convert_with_fallback(python_code, model_name, last_successful_code, last_successful_model):
    """Convert code with fallback functionality"""
    cpp_result, updated_last_successful_code, updated_last_successful_model, is_fallback, fallback_details = handle_model_conversion(
        python_code, model_name, last_successful_code, last_successful_model
    )
    
    # Generate status message 
    if cpp_result.startswith(f"// {model_name} conversion failed"):
        source_model = fallback_details.get("source_model", "previous model")
        status_content = f"⚠️ **{model_name} conversion failed** - Using fallback from {source_model}"
        status_update = gr.update(value=status_content, visible=True)
    elif cpp_result.startswith(f"// {model_name} conversion error"):
        source_model = fallback_details.get("source_model", "previous model")
        status_content = f"❌ **{model_name} conversion error** - Using fallback from {source_model}"  
        status_update = gr.update(value=status_content, visible=True)
    else:
        status_update = gr.update(value="", visible=False)  # Hide for successful conversions
    
    return cpp_result, updated_last_successful_code, updated_last_successful_model, is_fallback, fallback_details, status_update

with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++")
    
    # Initialize results state
    results_state = gr.State([
        ["GPT", "", ""], 
        ["Claude", "", ""],
        ["Gemini", "", ""]
    ])
    
    # Track last successful C++ conversion for fallback
    last_successful_code = gr.State("")
    last_successful_model = gr.State("")
    
    # Track current fallback status
    is_using_fallback = gr.State(False)
    fallback_info = gr.State({"failed_model": "", "source_model": ""})
    
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=python_hard, lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude", "Gemini"], label="Select model", value="GPT")
    with gr.Row():
        convert = gr.Button("Convert code")
    
    # Status message for conversion results
    status_msg = gr.Markdown("", visible=False, elem_classes=["status-message"])
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++")
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="C++ result:", elem_classes=["cpp"])
    
    # Performance comparison table
    gr.Markdown("## Performance Comparison")
    results_table = gr.Dataframe(
        headers=["Model", "Python Time", "C++ Time"],
        datatype=["str", "str", "str"],
        value=[
            ["GPT", "", ""], 
            ["Claude", "", ""],
            ["Gemini", "", ""]
        ],
        label="Execution Times",
        interactive=False,
        wrap=True
    )

    # Event handlers
    convert.click(
        convert_with_fallback, 
        inputs=[python, model, last_successful_code, last_successful_model], 
        outputs=[cpp, last_successful_code, last_successful_model, is_using_fallback, fallback_info, status_msg]
    )
    
    python_run.click(execute_python, inputs=[python], outputs=[python_out]).then(
        update_python_results, 
        inputs=[python_out, model, results_state], 
        outputs=[python_out, results_state]
    ).then(
        lambda results: results,
        inputs=[results_state],
        outputs=[results_table]
    )
    
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out]).then(
        update_cpp_results,
        inputs=[cpp_out, model, results_state, is_using_fallback, fallback_info],
        outputs=[cpp_out, results_state]
    ).then(
        lambda results: results,
        inputs=[results_state], 
        outputs=[results_table]
    )

ui.launch(inbrowser=True)

In [ ]:
# Replace this with the right C++ compile + execute command for your platform

!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
!./optimized


In [ ]:
# Replace this with the right C++ compile + execute command for your platform

!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
!./optimized
